idée sujet :

rechercher les partenaires privilégiés : par exemple regarder sur des acteurs précis combien de value ils échangent avec certains partenaires avec lequels ils ont plus d'échanges ( au cours du temps) => essayer de regarder si des préférent certains partenaires

on peut chercher ceux du plus gros acteurs déterminés dans profit analysis

In [67]:
import glob
from datetime import datetime
import pandas as pd
from os import path

fileNames = glob.glob('networks/*')
print(fileNames)
file_dict = {
    datetime.strptime(path.basename(file).split('.')[0], '%Y-%m-%d'): pd.read_csv(file)
    for file in fileNames
}

compA = pd.concat(file_dict).sort_index()
compA

['networks\\2015-1-1.csv', 'networks\\2015-1-10.csv', 'networks\\2015-1-11.csv', 'networks\\2015-1-12.csv', 'networks\\2015-1-13.csv', 'networks\\2015-1-14.csv', 'networks\\2015-1-15.csv', 'networks\\2015-1-16.csv', 'networks\\2015-1-17.csv', 'networks\\2015-1-18.csv', 'networks\\2015-1-19.csv', 'networks\\2015-1-2.csv', 'networks\\2015-1-20.csv', 'networks\\2015-1-21.csv', 'networks\\2015-1-22.csv', 'networks\\2015-1-23.csv', 'networks\\2015-1-24.csv', 'networks\\2015-1-25.csv', 'networks\\2015-1-26.csv', 'networks\\2015-1-27.csv', 'networks\\2015-1-28.csv', 'networks\\2015-1-29.csv', 'networks\\2015-1-3.csv', 'networks\\2015-1-30.csv', 'networks\\2015-1-31.csv', 'networks\\2015-1-4.csv', 'networks\\2015-1-5.csv', 'networks\\2015-1-6.csv', 'networks\\2015-1-7.csv', 'networks\\2015-1-8.csv', 'networks\\2015-1-9.csv', 'networks\\2015-10-1.csv', 'networks\\2015-10-10.csv', 'networks\\2015-10-11.csv', 'networks\\2015-10-12.csv', 'networks\\2015-10-13.csv', 'networks\\2015-10-14.csv', 'net

Source        Target        value  \
2015-01-01 0     ePay.info_CoinJoinMess  CloudBet.com      3519173   
           1                     Cex.io          1956      8491196   
           2                     157228     C-Cex.com  10833480021   
           3                          0           421     27123449   
           4               BTCGuild.com        787953   2507884460   
...                                 ...           ...          ...   
2017-06-30 5463                71554939           227    381902496   
           5464                72288955            44      1280939   
           5465                 4833806           386   1192762366   
           5466                 1228047           189   1080246068   
           5467                      94     BTC-e.com    194944382   

                 nb_transactions  
2015-01-01 0                   1  
           1                   3  
           2                   1  
           3                   8  
           4                   1  
...                          ...  
2017-06-30 5463                1  
           5464                1  
           5465                1  
           5466                2  
           5467                3  

[5876734 rows x 4 columns]

In [76]:
# keep only 2015/01/01
compA2015 = compA.loc['2015-01-01']

# make a df of the actors and an edge list of their target

adjacency_list = compA2015.groupby('Source').agg({'Target': lambda x: list(x), 'value': lambda x: list(x)})

# make the index a column
adjacency_list.reset_index(inplace=True)

# set the index to a list of numbers
adjacency_list.index = range(len(adjacency_list))


adjacency_list

,Source,Target,value
0,0,"[421, 1323, 1956, NitrogenSports.eu, ePay.info...","[27123449, 4347764, 400114, 1214270, 70585962,..."
1,100157,[27723],[8378114]
2,10073457,"[72209845, 74330500, CoinTrader.net_LocalBitco...","[87120857, 90926607, 76740857]"
3,10166,[72446410],[361010000]
4,10251,[CoinTrader.net_LocalBitcoins.com],[11316835]
...,...,...,...
517,VirWoX.com,[342],[31210824576]
518,Vircurex.com,"[0, 2024]","[7999012, 9960000]"
519,YABTCL.com,[0],[675207289]
520,Zyado.com,"[Kraken.com, Bitcoin.de]","[173904000, 206500000]"


In [75]:
# plot the graph with plotly

import plotly.graph_objects as go
import networkx as nx

G = nx.from_pandas_edgelist(compA2015, 'Source', 'Target', 'value', create_using=nx.DiGraph())

# set the positions of the nodes with random values
import random
for node in G.nodes():
    G.nodes[node]['pos'] = [random.random(), random.random()]
    

print(G.nodes())
print(G.edges())

edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))


# plot the graph with plotly

import plotly.graph_objects as go

node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append('# of connections: '+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Network graph made with Python',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()


['ePay.info_CoinJoinMess', 'CloudBet.com', 'Cex.io', '1956', '157228', 'C-Cex.com', '0', '421', 'BTCGuild.com', '787953', 'SatoshiMines.com', 'Bitcoin.de', 'Cryptsy.com', '72236394', '16425460', '72603512', '71554939', 'BTCJam.com', 'Bitfinex.com', '72472408', '72995324', '1748906', 'BTC-e.com', 'Hashnest.com', 'OkLink.com', '8539', '18972', 'SpectroCoin.com', '1323', 'AnoniBet.com', 'CoinTrader.net_LocalBitcoins.com', '73021966', 'NitrogenSports.eu', '144727', '1647', '31317920', '56717', '1285711', '70482234', 'Bitstamp.net', '70822184', '1294', '69251458', '1226', 'BtcTrade.com', '1487245', '4273', 'Kraken.com', '3226874', '7328', '57', '978', 'FoxBit.com.br', 'MercadoBitcoin.com.br', '58489', 'BitPay.com', 'Poloniex.com', 'Cryptonator.com', '72102457', '19643031', '2624', '47678', '2545', '2566', '289', '73846409', 'Bittrex.com', '107', 'Loanbase.com', 'BX.in.th', '536', 'Luno.com', 'SlushPool.com', 'BTCCPool', 'Cavirtex.com', '72209869', '71424456', 'Purse.io', 'AllCoin.com', 'Fau